## 层和块

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1824,  0.0462, -0.0796,  0.0964,  0.0096,  0.0326,  0.0245, -0.0101,
         -0.1823,  0.3415],
        [-0.2932,  0.0292,  0.0784,  0.0151,  0.0917,  0.0890,  0.0824, -0.0978,
         -0.1966,  0.1227]], grad_fn=<AddmmBackward0>)

### 自定义块

In [4]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))
    
net = MLP()
net(X)

tensor([[ 0.0920,  0.0874,  0.2266, -0.0318,  0.2412,  0.1384, -0.1270, -0.2524,
          0.0642,  0.0653],
        [ 0.1274, -0.0818,  0.1948, -0.1712,  0.1896,  0.1992, -0.1342, -0.2669,
          0.1125,  0.1131]], grad_fn=<AddmmBackward0>)

### 顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0368,  0.1458, -0.0167,  0.0022,  0.0300,  0.2000,  0.0609,  0.1718,
          0.1044,  0.2482],
        [-0.0538,  0.2616, -0.0085, -0.1604,  0.0127,  0.0613, -0.2458,  0.0647,
         -0.0370,  0.1184]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0802, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.4133, grad_fn=<SumBackward0>)